## Data Cleaning

In [1]:
# load necessary libraries
import numpy as np
import pandas as pd

In [60]:
import warnings
warnings.filterwarnings('ignore')

### Tweets

In [159]:
# load data sets

tweets_2020 = pd.read_csv('data/needed/raw/2020.csv', index_col=0)
tweets_2021 = pd.read_csv('data/needed/raw/2021.csv')
tweets_2022 = pd.read_csv('data/needed/raw/2022.csv')

In [160]:
# tweets_2022 contains duplicates with very slightly different likes, replies and retweet counts; we remove them
tweets_2022 = tweets_2022.drop_duplicates(subset=["id"], keep="last")

In [161]:
# find differences in columns

print(set(tweets_2020)-(set(tweets_2021)))
print(set(tweets_2021)-(set(tweets_2020)))
print(set(tweets_2021).symmetric_difference(set(tweets_2022)))

{'user_id_str', 'hour', 'nlikes', 'search', 'nreplies', 'nretweets', 'day'}
{'likes_count', 'time', 'retweets_count', 'replies_count', 'mentions'}
set()


In [162]:
# make column names match between dataframes

tweets_2020 = tweets_2020.rename(columns = {"day":"day_of_week", "hour":"hour_of_day"})

tweets_2021 = tweets_2021.rename(columns = {"retweets_count":"nretweets", "replies_count":"nreplies", "likes_count":"nlikes"})
tweets_2022 = tweets_2022.rename(columns = {"retweets_count":"nretweets", "replies_count":"nreplies", "likes_count":"nlikes"})

In [163]:
# make one column with date and time
tweets_2021["date"] = tweets_2021["date"] + " " + tweets_2021["time"]
tweets_2021 = tweets_2021.drop(["time"], 1)

tweets_2022["date"] = tweets_2022["date"] + " " + tweets_2022["time"]
tweets_2022 = tweets_2022.drop(["time"], 1)

# make the date columns as date in pandas
for i in [tweets_2020, tweets_2021, tweets_2022]:
    i["date"] = pd.to_datetime(i["date"])

In [164]:
# add columns that were in one data frame but not the others

tweets_2021["day_of_week"] = tweets_2021["date"].dt.dayofweek + 1
tweets_2022["day_of_week"] = tweets_2022["date"].dt.dayofweek + 1

tweets_2021["hour_of_day"] = tweets_2021["date"].dt.hour
tweets_2022["hour_of_day"] = tweets_2022["date"].dt.hour


In [165]:
# check again differences in columns

print(set(tweets_2020)-(set(tweets_2021)))
print(set(tweets_2021)-(set(tweets_2020)))
print(set(tweets_2021).symmetric_difference(set(tweets_2022)))

{'user_id_str', 'search'}
{'mentions'}
set()


In [166]:
print(tweets_2020.shape)
print(tweets_2021.shape)
print(tweets_2022.shape)

(11717, 38)
(3115, 37)
(514, 37)


In [167]:
# make one df out of all 3 datasets
tweets = tweets_2020.append(tweets_2021, ignore_index= True)
tweets = tweets.append(tweets_2022, ignore_index= True)
print(tweets.shape)
tweets = tweets.sort_values(by=['date'], ascending=False)

(15346, 39)


In [168]:
# remove all empty columns

#tweets.info()
tweets = tweets.dropna(1, how="all")
#tweets.info()

In [169]:
# save tweets df to csv
tweets.to_csv('data/needed/cleaned/Tweets.csv', index=False)

### Stock Price

In [170]:
# load data sets

stock_2018 = pd.read_csv('data/needed/raw/TSLA_2010_2018.csv')
stock_2022 = pd.read_csv('data/needed/raw/TSLA_2017_2022.csv')


In [171]:
# since "Adj Close" os always the same as "Close" we drop it
stock_2018 = stock_2018.drop(["Adj Close"], 1)

# rename "Close/Last" to "Close" to merge df's
stock_2022 = stock_2022.rename(columns = {"Close/Last":"Close"})

In [172]:
# make the date columns as date in pandas
stock_2022["Date"] = pd.to_datetime(stock_2022["Date"], format="%m/%d/%Y")
stock_2018["Date"] = pd.to_datetime(stock_2018["Date"], format="%Y-%m-%d")

In [173]:
print(stock_2018.shape)
print(stock_2022.shape)

(2056, 6)
(1260, 6)


In [174]:
# put the 2 df's together and remove all duplicates
stock = stock_2018.append(stock_2022, ignore_index=True)
print(stock.shape)

stock = stock.drop_duplicates(subset=["Date"])
print(stock.shape)

(3316, 6)
(2957, 6)


In [175]:
# save stock df to csv
stock = stock.sort_values(by=['Date'], ascending=False)
stock.to_csv('data/needed/cleaned/Stock.csv', index=False)